In [ ]:
!pip install beautifulsoup4 requests

In [1]:
import requests
from bs4 import BeautifulSoup

# The URL of the webpage you want to scrape
url = 'https://www.booking.com/hotel/be/hydro-palace.en-gb.html?label=gen173nr-1FCA0oFUIMaHlkcm8tcGFsYWNlSDNYBGhsiAEBmAEJuAEHyAEN2AEB6AEB-AELiAIBqAIDuALtqdeHBsACAdICJGVhMGRkOWQ3LTUxOTctNDZjMC1hMDU2LTRlYTIxMTJhMDZhONgCBuACAQ;sid=112feb5c809e55c93c7a10cb30d87aae'

# Send a GET request to the webpage
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the webpage content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the image tag with the specific attributes
    image_tag = soup.find('img', class_='hide')

    if image_tag:
        # Extract the image URL
        image_url = image_tag['src']



        print(image_url)


    else:
        print("Image tag not found.")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


https://cf.bstatic.com/xdata/images/hotel/max1024x768/140334767.jpg?k=ea5de36f51271d061667d17181c452063d8ccd25c8e1b96f22ea231b5d11fce5&o=&hp=1


In [30]:
def scrapping_image_and_location(url):
    # Send a GET request to the webpage
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the webpage content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the image tag with the specific attributes
        image_tag = soup.find('img', class_='hide')
        if image_tag:
            # Extract the image URL
            image_url = image_tag['src']
        else:
            image_url = None
            print("Image tag not found.")

        # Find the span tag with the specific classes for location
        location_tag = soup.find('span', class_='hp_address_subtitle js-hp_address_subtitle jq_tooltip')
        if location_tag:
            # Extract the location text
            location = location_tag.get_text(strip=True)
        else:
            location = None
            print("Location tag not found.")

        return image_url, location
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return None, None

url = 'https://www.booking.com/hotel/be/hydro-palace.en-gb.html?label=gen173nr-1FCA0oFUIMaHlkcm8tcGFsYWNlSDNYBGhsiAEBmAEJuAEHyAEN2AEB6AEB-AELiAIBqAIDuALtqdeHBsACAdICJGVhMGRkOWQ3LTUxOTctNDZjMC1hMDU2LTRlYTIxMTJhMDZhONgCBuACAQ;sid=112feb5c809e55c93c7a10cb30d87aae'
image_url, location = scrapping_image_and_location(url)
print(f"Image URL: {image_url}")
print(f"Location: {location}")



('https://cf.bstatic.com/xdata/images/hotel/max1024x768/140334767.jpg?k=ea5de36f51271d061667d17181c452063d8ccd25c8e1b96f22ea231b5d11fce5&o=&hp=1',
 'Zeedijk 250 bus 17, Mariakerke, 8400 Ostend, Belgium')

# Dataset import

### installing

In [2]:

! pip install kaggle


### setup

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [11]:
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/kaggle.json

In [12]:

! chmod 600 ~/.kaggle/kaggle.json

In [13]:
!kaggle datasets download -d thedevastator/booking-com-hotel-reviews

Dataset URL: https://www.kaggle.com/datasets/thedevastator/booking-com-hotel-reviews
License(s): other
booking-com-hotel-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [14]:
! unzip booking-com-hotel-reviews.zip

Archive:  booking-com-hotel-reviews.zip
replace booking_reviews copy.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: booking_reviews copy.csv  


# explore dataset

### exploring

In [15]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import re

In [16]:

db_reviews = pd.read_csv("booking_reviews copy.csv")
db_reviews.head()

,index,review_title,reviewed_at,reviewed_by,images,crawled_at,url,hotel_name,hotel_url,avg_rating,nationality,rating,review_text,raw_review_text,tags,meta
0,0,Exceptional,11 July 2021,Kyrylo,NaN,"07/20/2021, 01:09:27",https://www.booking.com/reviews/be/hotel/villa...,Villa Pura Vida,https://www.booking.com/hotel/be/villa-pura-vi...,9.7,Poland,10.0,"Everything was perfect! Quite, cozy place to r...","<p class=""review_pos ""><svg aria-label=""Positi...",Business trip~Solo traveller~Junior Suite~Stay...,"{'language': 'en-gb', 'source': 'https://www.b..."
1,1,I highly recommend this b&b! We enjoyed it a lot!,24 November 2019,Dimitri,NaN,"07/20/2021, 01:09:27",https://www.booking.com/reviews/be/hotel/villa...,Villa Pura Vida,https://www.booking.com/hotel/be/villa-pura-vi...,9.7,Belgium,9.0,Very friendly host and perfect breakfast!,"<p class=""review_pos ""><svg aria-label=""Positi...",Leisure trip~Couple~Deluxe Suite~Stayed 1 nigh...,"{'language': 'en-gb', 'source': 'https://www.b..."
2,2,Exceptional,3 January 2020,Virginia,NaN,"07/20/2021, 01:09:27",https://www.booking.com/reviews/be/hotel/hydro...,Hydro Palace Apartment,https://www.booking.com/hotel/be/hydro-palace....,9.2,United Kingdom,10.0,It was just what we wanted for a week by the b...,"<p class=""review_neg ""><svg aria-label=""Negati...",Leisure trip~Couple~Apartment with Sea View~St...,"{'language': 'en-gb', 'source': 'https://www.b..."
3,3,My stay in the house was a experiencing bliss ...,8 September 2019,Kannan,NaN,"07/20/2021, 01:09:28",https://www.booking.com/reviews/be/hotel/villa...,Villa Pura Vida,https://www.booking.com/hotel/be/villa-pura-vi...,9.7,Netherlands,10.0,My stay in the house was a experiencing bliss ...,"<p class=""review_pos ""><svg aria-label=""Positi...",Business trip~Solo traveller~Junior Suite~Stay...,"{'language': 'en-gb', 'source': 'https://www.b..."
4,4,One bedroom apartment with wonderful view and ...,23 June 2019,Sue,https://cf.bstatic.com/xdata/images/xphoto/squ...,"07/20/2021, 01:09:28",https://www.booking.com/reviews/be/hotel/hydro...,Hydro Palace Apartment,https://www.booking.com/hotel/be/hydro-palace....,9.2,South Africa,9.2,The building itself has a very musty smell in ...,"<p class=""review_neg ""><svg aria-label=""Negati...",Leisure trip~People with friends~Apartment wit...,"{'language': 'en-gb', 'source': 'https://www.b..."


In [17]:
db_reviews.columns

Index(['index', 'review_title', 'reviewed_at', 'reviewed_by', 'images',
       'crawled_at', 'url', 'hotel_name', 'hotel_url', 'avg_rating',
       'nationality', 'rating', 'review_text', 'raw_review_text', 'tags',
       'meta'],
      dtype='object')

In [24]:
required_columns=['hotel_name','hotel_url',	'avg_rating']
#filter the dataset to include only the required columns
df=db_reviews[required_columns]
df.head()

,hotel_name,hotel_url,avg_rating
0,Villa Pura Vida,https://www.booking.com/hotel/be/villa-pura-vi...,9.7
1,Villa Pura Vida,https://www.booking.com/hotel/be/villa-pura-vi...,9.7
2,Hydro Palace Apartment,https://www.booking.com/hotel/be/hydro-palace....,9.2
3,Villa Pura Vida,https://www.booking.com/hotel/be/villa-pura-vi...,9.7
4,Hydro Palace Apartment,https://www.booking.com/hotel/be/hydro-palace....,9.2


In [25]:
df.shape

(26675, 3)

In [26]:
df = df.dropna(subset=['hotel_name', 'hotel_url'])


In [27]:
df.shape

(26386, 3)

In [28]:
df= df.drop_duplicates(subset=['hotel_name', 'hotel_url', 'avg_rating'])
df.shape
df

,hotel_name,hotel_url,avg_rating
0,Villa Pura Vida,https://www.booking.com/hotel/be/villa-pura-vi...,9.7
2,Hydro Palace Apartment,https://www.booking.com/hotel/be/hydro-palace....,9.2
6,Love Nest Versailles - Unique Louis XIV decora...,https://www.booking.com/hotel/be/love-nest-ver...,8.8
7,Gite Namaste,https://www.booking.com/hotel/be/gite-namaste....,9.3
12,Hotel Saint Sauveur by WP Hotels,https://www.booking.com/hotel/be/stsauveur.en-...,7.8
...,...,...,...
26424,Smartflats City - Royal,https://www.booking.com/hotel/be/smartflats-ro...,7.6
26573,Glamping op het Zeugekot,https://www.booking.com/hotel/be/glamping-op-h...,8.6
26575,Hotel Bilderdijk,https://www.booking.com/hotel/be/bilderdijk.en...,9.0
26576,B&B Roosendaelhof,https://www.booking.com/hotel/be/roosendaelhof...,8.3


In [32]:
#df['image_url'],df['location'] = df.hotel_url.apply(lambda url: scrapping_image_and_location(url))

print(df.shape[0])
image_urls = []
locations=[]
for j in range(0, df.shape[0]):
  print(j)
  image_url, location=scrapping_image_and_location(df.iloc[j]["hotel_url"])
  image_urls.append(image_url)
  locations.append(location)

image_urls
locations

822
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
Image tag not found.
Location tag not found.
17
18
19
20
21
Image tag not found.
Location tag not found.
22
23
24
25
26
27
28
29
30
31
32
Image tag not found.
Location tag not found.
33
Image tag not found.
Location tag not found.
34
35
36
37
38
Image tag not found.
Location tag not found.
39
40
41
42
43
44
Image tag not found.
Location tag not found.
45
46
47
48
49
50
51
52
Image tag not found.
Location tag not found.
53
54
Image tag not found.
Location tag not found.
55
Image tag not found.
Location tag not found.
56
57
58
Image tag not found.
Location tag not found.
59
60
61
62
63
64
65
66
Image tag not found.
Location tag not found.
67
68
69
70
Image tag not found.
Location tag not found.
71
72
73
74
75
76
77
78
79
80
81
Image tag not found.
Location tag not found.
82
83
84
Image tag not found.
Location tag not found.
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
Image tag not found.
Location tag not found.
105
106
107

['Sint-Denijsestraat 191C, 8500 Kortrijk, Belgium',
 'Zeedijk 250 bus 17, Mariakerke, 8400 Ostend, Belgium',
 '39 Rue des Croisiers, 5000 Namur, Belgium',
 '150 Quartier du Pairet, 5680 Gochenée, Belgium',
 'Langestraat 50, 8370 Blankenberge, Belgium',
 'lindenstraat 7A, 9300 Aalst, Belgium',
 'Hoogpoort 52, Binnenstad, 9000 Ghent, Belgium',
 'Rue Général Leman 97, Etterbeek, 1040 Brussels, Belgium',
 'Boulevard Adolphe Max 98, 1000 Brussels, Belgium',
 'Achilles Musschestraat 39, Stationsbuurt-Zuid, 9000 Ghent, Belgium',
 'Rue de Belle-Vue, 126/128, 7100 La Louvière, Belgium',
 "Rue Comte d'Ursel 19, 6940 Durbuy, Belgium",
 'Rue Stéphanie 143, Laeken, 1020 Brussels, Belgium',
 '61 rue saules bastin, 4920 Aywaille, Belgium',
 '4920 Aywaille, Belgium',
 '271 Avenue Louise, Elsene / Ixelles, 1050 Brussels, Belgium',
 None,
 '30 Place du Marché, 4000 Liège, Belgium',
 'Noorderlaan 100, Antwerp District, 2030 Antwerp, Belgium',
 'Moerdijkbeek Oost 28, 8680 Koekelare, Belgium',
 'Rue Régnié

In [33]:
df['image_url']=image_urls
df['location']=locations
df.head()

,hotel_name,hotel_url,avg_rating,image_url,location
0,Villa Pura Vida,https://www.booking.com/hotel/be/villa-pura-vi...,9.7,https://cf.bstatic.com/xdata/images/hotel/max1...,"Sint-Denijsestraat 191C, 8500 Kortrijk, Belgium"
2,Hydro Palace Apartment,https://www.booking.com/hotel/be/hydro-palace....,9.2,https://cf.bstatic.com/xdata/images/hotel/max1...,"Zeedijk 250 bus 17, Mariakerke, 8400 Ostend, B..."
6,Love Nest Versailles - Unique Louis XIV decora...,https://www.booking.com/hotel/be/love-nest-ver...,8.8,https://cf.bstatic.com/xdata/images/hotel/max1...,"39 Rue des Croisiers, 5000 Namur, Belgium"
7,Gite Namaste,https://www.booking.com/hotel/be/gite-namaste....,9.3,https://cf.bstatic.com/xdata/images/hotel/max1...,"150 Quartier du Pairet, 5680 Gochenée, Belgium"
12,Hotel Saint Sauveur by WP Hotels,https://www.booking.com/hotel/be/stsauveur.en-...,7.8,https://cf.bstatic.com/xdata/images/hotel/max1...,"Langestraat 50, 8370 Blankenberge, Belgium"


In [34]:
df.to_csv("Hotel_Booking_dataset.csv",index=True)